In [1]:
from ase.build import bulk
import pyscal.core as pc
from ase import Atom, Atoms
import numpy as np
import pyscal.traj_process as ptp
from ase.calculators.lammpsrun import LAMMPS
from ase.io import write

In [2]:
sys = pc.System()
sys.read_inputfile("conf.triclinic.dump")

In [3]:
sys.calculate_energy(species=['Mo'], pair_style='eam/alloy', pair_coeff='* * Mo.set Mo', mass=95)

In [5]:
sys.atoms[10].energy

-5.999615135082189

In [6]:
sys.atoms[10].id

11

In [3]:
sys.to_file('test.data', format='loco', species=['Mo'])

In [4]:
asesys = ptp.convert_to_ase(sys, species=["Mo"])

In [5]:
write("n1.dump", asesys, format="lammps-data")

In [6]:
parameters = {'pair_style': 'eam/alloy',
              'pair_coeff': '* * Mo.set Mo'}

In [22]:
from lammps import lammps

In [23]:
L = lammps()

In [24]:
L.command("echo log")
L.command("units metal")
L.command("atom_style atomic")
L.command("boundary p p p")

In [25]:
L.command('read_data n1.dump')
L.command('pair_style %s'%parameters["pair_style"])
L.command('pair_coeff %s'%parameters["pair_coeff"])
L.command("mass * %f"%95)

In [26]:
L.command("compute 1 all pe/atom")

In [40]:
L.command("dump 1 all custom 1 traj.dat id type x y z c_1")
L.command("run 0")

In [35]:
xx = L.extract_atom("id",0)

In [39]:
xx[2]

243

In [15]:
eng = L.extract_compute("1",1,1)

In [17]:
from ctypes import c_double, c_int

In [41]:
eng[0]

-6.091159522695859

In [21]:
L.close()